In [5]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
import pandas as pd
import re
import time
import os

In [ ]:
#content > div.section.inner_sub > table.type2 > tbody > tr:nth-child(7) > td.title > a

In [94]:
def Crawler(x_code, x_page):
    # start = time.time()
    df = pd.DataFrame(columns = [
    'date',
    'title',
    'content',
    'click',
    'good',
    'bad'
    ])
    # 마지막 페이지
    url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, 1)
    html = requests.get(url, headers = headers).text
    soup = bs(html, 'html.parser')
    last_page = soup.select('#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgRR ')[0]
    last_page = last_page.find('a')
    last_page = last_page.attrs['href'][33:]
    last_page = int(last_page)
    # print(last_page)

    if page == 'all':
        for p in range(1, last_page+1):
            url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, p)
            html = requests.get(url, headers = headers).text
            soup = bs(html, 'html.parser')
            base = soup.select('#content > div.section.inner_sub > table.type2 > tbody')[0]
            
            for i in range(3, 26):
                # 5칸마다 있는 줄에 해당할 경우 contunue
                try:
                    if i == 8 or i == 14 or i == 20:
                        continue

                    date = base.select('tr:nth-child({}) > td:nth-child(1) > span'.format(i))[0].text
                except:
                    break

                title = base.select('tr:nth-child({}) > td.title > a'.format(i))[0].text
                click = base.select('tr:nth-child({}) > td:nth-child(4) > span'.format(i))[0].text
                good = base.select('tr:nth-child({}) > td:nth-child(5) > strong'.format(i))[0].text
                bad = base.select('tr:nth-child({}) > td:nth-child(6) > strong'.format(i))[0].text

                # href 얻기
                href = base.select('tr:nth-child({}) > td.title'.format(i))[0]
                href = href.find('a')
                href = href.attrs['href']

                # 본문 내용 얻기
                content_url = ('https://finance.naver.com'+ href)
                content_html = requests.get(content_url, headers = headers).text
                content_soup = bs(content_html, 'html.parser')
                content = content_soup.find("div", "view_se").text

                # df에 저장
                result = {
                    'date':date,
                    'title':title,
                    'content':content,
                    'click':click,
                    'good':good,
                    'bad':bad
                    }
                df = df.append(result, ignore_index = True)
                #end = time.time()
                #print("경과 시간: {}".format(end - start))
    else:
        if last_page < x_page:
            x_page = last_page   
            
        for p in range(1, x_page+1):
            url = "https://finance.naver.com/item/board.nhn?code={}&page={}".format(x_code, p)
            html = requests.get(url, headers = headers).text
            if p >= 2:
                if html_check == html:
                    break
            soup = bs(html, 'html.parser')
            base = soup.select('#content > div.section.inner_sub > table.type2 > tbody')[0]

            for i in range(3, 26):
                # 5칸마다 있는 줄에 해당할 경우 contunue
                try:
                    if i == 8 or i == 14 or i == 20:
                        continue

                    date = base.select('tr:nth-child({}) > td:nth-child(1) > span'.format(i))[0].text
                except:
                    break

                title = base.select('tr:nth-child({}) > td.title > a'.format(i))[0].text
                click = base.select('tr:nth-child({}) > td:nth-child(4) > span'.format(i))[0].text
                good = base.select('tr:nth-child({}) > td:nth-child(5) > strong'.format(i))[0].text
                bad = base.select('tr:nth-child({}) > td:nth-child(6) > strong'.format(i))[0].text

                # href 얻기
                href = base.select('tr:nth-child({}) > td.title'.format(i))[0]
                href = href.find('a')
                href = href.attrs['href']

                # 본문 내용 얻기
                content_url = ('https://finance.naver.com'+ href)
                content_html = requests.get(content_url, headers = headers).text
                content_soup = bs(content_html, 'html.parser')
                content = content_soup.find("div", "view_se").text

                # df에 저장
                result = {
                    'date':date,
                    'title':title,
                    'content':content,
                    'click':click,
                    'good':good,
                    'bad':bad
                    }
                df = df.append(result, ignore_index = True)
            #end = time.time()
            #print("경과 시간: {}".format(end - start))
    return df

In [95]:
# Hyper-parameters
code = '094170'
page = 200# 마지막 페이지까지 하고 싶으면 'all'
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}

In [96]:
#Crawling
df = Crawler(code, page)

161


KeyboardInterrupt: 

In [ ]:
df

In [6]:
# 결과 저장
df.to_excel('{}_result.xlsx'.format(code))